In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_daily_1 = pd.read_csv('Results/GRU/Daily_Final/GRU_final_daily(2)_test_predictions_model_30.csv')
df_daily_2 = pd.read_csv('Results/GRU/Daily_Final/GRU_final_daily(2)_test_predictions_model_26.csv')
df_daily_3 = pd.read_csv('Results/GRU/Daily_Final/GRU_final_daily(2)_test_predictions_model_12.csv')
df_daily_4 = pd.read_csv('Results/GRU/Daily_Final/GRU_final_daily(2)_test_predictions_model_33.csv')
df_daily_5 = pd.read_csv('Results/GRU/Daily_Final/GRU_final_daily(2)_test_predictions_model_11.csv')

df_4h_1 = pd.read_csv('Results/GRU/4h_Final/df4h_FINAL_test_predictions_model_35.csv')
df_4h_2 = pd.read_csv('Results/GRU/4h_Final/df4h_FINAL_test_predictions_model_29.csv')
df_4h_3 = pd.read_csv('Results/GRU/4h_Final/df4h_FINAL_test_predictions_model_21.csv')
df_4h_4 = pd.read_csv('Results/GRU/4h_Final/df4h_FINAL_test_predictions_model_26.csv')
df_4h_5 = pd.read_csv('Results/GRU/4h_Final/df4h_FINAL_test_predictions_model_28.csv')

df_30min_1 = pd.read_csv('Results/GRU/30m_Final/df30m_FINAL_test_predictions_model_25.csv')
df_30min_2 = pd.read_csv('Results/GRU/30m_Final/df30m_FINAL_test_predictions_model_36.csv')
df_30min_3 = pd.read_csv('Results/GRU/30m_Final/df30m_FINAL_test_predictions_model_26.csv')
df_30min_4 = pd.read_csv('Results/GRU/30m_Final/df30m_FINAL_test_predictions_model_15.csv')
df_30min_5 = pd.read_csv('Results/GRU/30m_Final/df30m_FINAL_test_predictions_model_21.csv')

df_4h_baseline = pd.read_csv('Results/LSTM/4H_GRID_LSTM/RUN1_4H_test_predictions_model_1.csv')
df_30min_baseline = pd.read_csv('Results/LSTM/30MIN_GRID_LSTM/RUN(1)_30MIN_test_predictions_model_1.csv')
df_daily_baseline = pd.read_csv('Results/LSTM/1D_GRID_LSTM/RUN1_DAILY_test_predictions_model_1.csv')

In [3]:
dfs_daily = [df_daily_1, df_daily_2, df_daily_3, df_daily_4, df_daily_5]
dfs_4h =[df_4h_1, df_4h_2, df_4h_3, df_4h_4, df_4h_5]
dfs_30min = [df_30min_1, df_30min_2, df_30min_3, df_30min_4, df_30min_5]

dfs = dfs_daily + dfs_4h + dfs_30min

# Rename prediction column
for df in dfs:
    df.rename(columns={df.columns[1]: 'Predicted'}, inplace=True)

# Update 'Actual' values for 4h DataFrames
for df in dfs_4h:
    df['Actual'] = df_4h_baseline['Actual']

# Update 'Actual' values for 30min DataFrames
for df in dfs_30min:
    df['Actual'] = df_30min_baseline['Actual']

# Update 'Actual' values for Daily DataFrames
for df in dfs_daily:
    df['Actual'] = df_daily_baseline['Actual']

# Adjust 'Predicted' values for all 4h DataFrames
for df in dfs_4h:
    df['Predicted'] = df['Actual'] + df['Predicted']

# Adjust 'Predicted' values for all 30min DataFrames
for df in dfs_30min:
    df['Predicted'] = df['Actual'] + df['Predicted']

# Adjust 'Predicted' values for all Daily DataFrames
for df in dfs_daily:
    df['Predicted'] = df['Actual'] + df['Predicted']

In [4]:
# This function creates a new column, 'Shifted_Predicted', which represents what the model predicts for the value at t+1
def align_predictions(df):
    df['Shifted_Predicted'] = df['Predicted'].shift(-1)
    return df


# This functions calculates the percentage change forecast for t+1
def generate_comparison_metrics(df):
    df['Percent_Change_Forecast'] = ((df['Shifted_Predicted'] - df['Actual']) / df['Actual']) * 100
    return df


# Helper function to evaluate how the model performs when a trading strategy is simulated
def evaluate_performance(initial_cash, final_value):
    net_profit_loss = final_value - initial_cash
    roi = (final_value - initial_cash) / initial_cash * 100

    print(f"Initial Cash: ${initial_cash:.2f}")
    print(f"Final Portfolio Value: ${final_value:.2f}")
    print(f"Net Profit/Loss: ${net_profit_loss:.2f}")
    print(f"ROI: {roi:.2f}%")

    return net_profit_loss, roi


# Function to generate buy signals. Signal generated when the predicted value is greater than 'threshold_1'  
# percentage above the current value
def generate_buy_signals(df, threshold_1):
    df['Buy_Signal'] = df['Percent_Change_Forecast'] > threshold_1
    return df

# Function to generate sell signals. Signal generated when the predicted value is greater than (negative) 'threshold_2'  
# percentage below the current value
def generate_sell_signals(df, threshold_2):
    df['Sell_Signal'] = df['Percent_Change_Forecast'] < -threshold_2
    return df

In [12]:
def simulate_trading_strategy(df, initial_cash=10000):
    position = 0  # Current position (number of units held)
    cash = initial_cash
    trades = []  # Track trade history with detailed info
    buy_index = None  # Track index of buy to calculate trade duration

    # Loop through the DataFrame to simulate the strategy using the buy and sell signals
    for i in range(len(df) - 1):
        # Buy if a buy signal is present and no position is currently held
        if df['Buy_Signal'].iloc[i] and position == 0:
            buy_price = df['Actual'].iloc[i]
            position = cash / buy_price
            cash = 0
            buy_index = i
            trades.append({'Type': 'Buy', 'Index': i, 'Price': buy_price})

        # Sell if a sell signal is present and a position is currently held
        elif df['Sell_Signal'].iloc[i] and position > 0:
            sell_price = df['Actual'].iloc[i]
            cash = position * sell_price
            position = 0
            sell_index = i
            profit_loss = sell_price - trades[-1]['Price']
            duration = sell_index - buy_index  # Calculate duration of the trade
            trades.append({
                'Type': 'Sell',
                'Index': i,
                'Price': sell_price,
                'Profit/Loss': profit_loss,
                'Duration': duration
            })

    # Calculate the final portfolio value
    final_value = cash + position * df['Actual'].iloc[-1]

    # Convert trades to DataFrame
    trades_df = pd.DataFrame(trades)
    
    # Return the final value and the trades DataFrame
    return final_value, trades_df


def calculate_average_performance(results, timeframe):
    net_profits = []
    rois = []
    
    for model in results[timeframe].values():
        net_profits.append(model['net_profit_loss'])
        rois.append(model['roi'])
    
    avg_net_profit = np.mean(net_profits)
    avg_roi = np.mean(rois)
    
    print(f"Average Performance for {timeframe} models:")
    print(f"Average Net Profit: ${avg_net_profit:.2f}")
    print(f"Average ROI: {avg_roi:.2f}%")
    
    return {'average_net_profit': avg_net_profit, 'average_roi': avg_roi}


def calculate_total_trades(results, timeframe, model_key):
    trades_df = results[timeframe][model_key]['trades']
    total_trades = len(trades_df[trades_df['Type'] == 'Sell'])  # Count completed trades (pairs of Buy/Sell)

    return total_trades

    
def calculate_average_trade_length(results, timeframe, model_key):
    trades_df = results[timeframe][model_key]['trades']
    trade_durations = trades_df[trades_df['Type'] == 'Sell']['Duration']
    average_length = trade_durations.mean() if not trade_durations.empty else 0
    
    return average_length

In [6]:
def visualise_results_from_dictionary(results, baseline_actual, timeframe, model_key, xlim=None, ylim=None):
    # Extract trade data from the results dictionary
    trades_df = results[timeframe][model_key]['trades']
    
    # Plot the actual values from the baseline data
    plt.figure(figsize=(14, 6))
    plt.plot(baseline_actual, label='Actual Price', color='blue')
    
    # Mark buy and sell points using their indices from the trade data
    buy_points = trades_df[trades_df['Type'] == 'Buy']
    sell_points = trades_df[trades_df['Type'] == 'Sell']

    plt.scatter(buy_points['Index'], baseline_actual.iloc[buy_points['Index']], 
                marker='^', color='green', label='Buy Signal', s=100)
    plt.scatter(sell_points['Index'], baseline_actual.iloc[sell_points['Index']], 
                marker='v', color='red', label='Sell Signal', s=100)
    
    # Set title and labels
    plt.title(f'Trading Strategy: {model_key} ({timeframe})')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    
    # Set x and y limits if specified
    if xlim is not None:
        plt.xlim(xlim)
    if ylim is not None:
        plt.ylim(ylim)
    
    # Display the plot
    plt.show()

In [7]:
# Initialise variables and dictionaries
initial_cash = 10000
threshold_1 = 0.5
threshold_2 = 0.5
results = {
    'daily': {},
    '4h': {},
    '30min': {}
}

# Process dataframes
for df in dfs:
    align_predictions(df)
    generate_comparison_metrics(df)
    generate_buy_signals(df, threshold_1)
    generate_sell_signals(df, threshold_2)

In [8]:
# Process daily models, print profit and ROI for each model, save results
for _, df in enumerate(dfs_daily, start=1):
    print(f"\nModel_{_}:")  # Print model label
    final_value, trades_df = simulate_trading_strategy(df, initial_cash)
    net_profit_loss, roi = evaluate_performance(initial_cash, final_value)
    
    results['daily'][f'model_{_}'] = {
        'final_value': final_value,
        'trades': trades_df,
        'net_profit_loss': net_profit_loss,
        'roi': roi
    }


Model_1:
Initial Cash: $10000.00
Final Portfolio Value: $146733.98
Net Profit/Loss: $136733.98
ROI: 1367.34%

Model_2:
Initial Cash: $10000.00
Final Portfolio Value: $143157.67
Net Profit/Loss: $133157.67
ROI: 1331.58%

Model_3:
Initial Cash: $10000.00
Final Portfolio Value: $150712.40
Net Profit/Loss: $140712.40
ROI: 1407.12%

Model_4:
Initial Cash: $10000.00
Final Portfolio Value: $144600.27
Net Profit/Loss: $134600.27
ROI: 1346.00%

Model_5:
Initial Cash: $10000.00
Final Portfolio Value: $137267.72
Net Profit/Loss: $127267.72
ROI: 1272.68%


In [9]:
# Process 30min models, print profit and ROI for each model, save results
for _, df in enumerate(dfs_30min, start=1):
    print(f"\nModel_{_}:")  # Print model label
    final_value, trades_df = simulate_trading_strategy(df, initial_cash)
    net_profit_loss, roi = evaluate_performance(initial_cash, final_value)
    
    results['30min'][f'model_{_}'] = {
        'final_value': final_value,
        'trades': trades_df,
        'net_profit_loss': net_profit_loss,
        'roi': roi
    }


Model_1:
Initial Cash: $10000.00
Final Portfolio Value: $5224268.40
Net Profit/Loss: $5214268.40
ROI: 52142.68%

Model_2:
Initial Cash: $10000.00
Final Portfolio Value: $5249657.42
Net Profit/Loss: $5239657.42
ROI: 52396.57%

Model_3:
Initial Cash: $10000.00
Final Portfolio Value: $5326200.80
Net Profit/Loss: $5316200.80
ROI: 53162.01%

Model_4:
Initial Cash: $10000.00
Final Portfolio Value: $4446013.22
Net Profit/Loss: $4436013.22
ROI: 44360.13%

Model_5:
Initial Cash: $10000.00
Final Portfolio Value: $4500660.42
Net Profit/Loss: $4490660.42
ROI: 44906.60%


In [10]:
# Process 4h models, print profit and ROI for each model, save results
for _, df in enumerate(dfs_4h, start=1):
    print(f"\nModel_{_}:")  # Print model label
    final_value, trades_df = simulate_trading_strategy(df, initial_cash)
    net_profit_loss, roi = evaluate_performance(initial_cash, final_value)
    
    results['4h'][f'model_{_}'] = {
        'final_value': final_value,
        'trades': trades_df,
        'net_profit_loss': net_profit_loss,
        'roi': roi
    }


Model_1:
Initial Cash: $10000.00
Final Portfolio Value: $933463.28
Net Profit/Loss: $923463.28
ROI: 9234.63%

Model_2:
Initial Cash: $10000.00
Final Portfolio Value: $885520.14
Net Profit/Loss: $875520.14
ROI: 8755.20%

Model_3:
Initial Cash: $10000.00
Final Portfolio Value: $892213.14
Net Profit/Loss: $882213.14
ROI: 8822.13%

Model_4:
Initial Cash: $10000.00
Final Portfolio Value: $874721.54
Net Profit/Loss: $864721.54
ROI: 8647.22%

Model_5:
Initial Cash: $10000.00
Final Portfolio Value: $823436.90
Net Profit/Loss: $813436.90
ROI: 8134.37%


In [11]:
# Calculate average performances
avg_performance_4h = calculate_average_performance(results, '4h')
print('\n')
avg_performance_30min = calculate_average_performance(results, '30min')
print('\n')
avg_performance_daily = calculate_average_performance(results, 'daily')
print('\n')

Average Performance for 4h models:
Average Net Profit: $871871.00
Average ROI: 8718.71%


Average Performance for 30min models:
Average Net Profit: $4939360.05
Average ROI: 49393.60%


Average Performance for daily models:
Average Net Profit: $134494.41
Average ROI: 1344.94%




In [13]:
def calculate_average_across_models(results, timeframe):
    model_keys = [f"model_{i}" for i in range(1, 6)]  # List of model keys: model_1, model_2, ..., model_5
    total_trades_list = []
    average_trade_length_list = []

    # Iterate over each model key and calculate the metrics
    for model_key in model_keys:
        total_trades = calculate_total_trades(results, timeframe, model_key)
        average_length = calculate_average_trade_length(results, timeframe, model_key)
        
        total_trades_list.append(total_trades)
        average_trade_length_list.append(average_length)

    # Calculate averages
    average_total_trades = sum(total_trades_list) / len(total_trades_list)
    average_trade_length = sum(average_trade_length_list) / len(average_trade_length_list)

    print(f"Average Total Trades across models for {timeframe}: {average_total_trades:.2f}")
    print(f"Average Trade Length across models for {timeframe}: {average_trade_length:.2f} periods")

    return average_total_trades, average_trade_length


In [14]:
average_trades, average_length = calculate_average_across_models(results, '30min')
print('\n')
average_trades, average_length = calculate_average_across_models(results, '4h')
print('\n')
average_trades, average_length = calculate_average_across_models(results, 'daily')

Average Total Trades across models for 30min: 336.40
Average Trade Length across models for 30min: 17.83 periods


Average Total Trades across models for 4h: 146.60
Average Trade Length across models for 4h: 5.79 periods


Average Total Trades across models for daily: 41.60
Average Trade Length across models for daily: 3.73 periods
